## Mignotte's Secret Sharing

In [1]:
import random
import numpy as np
from functools import reduce
from math import sqrt

In [2]:
def generate_primes(start,n):
    '''
    params- start:The first prime in the generated mignotte sequence, 
            n:The number of primes in the mignotte sequence
    returns- A list of primes (potentially a mignotte sequence)
    '''
    mignotte = []
    primes=[2]
    p=3
    while(len(mignotte)<n):
        stop_iter=sqrt(p)
        i=0
        flag=True
        while(primes[i]<=stop_iter):
            if(p%primes[i]==0):
                flag=False
                break
            elif(i>=len(primes)):
                break
            i+=1
            
        if(flag):
            primes.append(p)
            if(p>start):
                mignotte.append(p)
        p+=2
    return(mignotte)

def check_mignotte_sequence(n,k,mignotte_seq):
    '''
    params- n:The number of participants, 
            k:the number of available shares, 
            mignotte_seq: the potential mignotte sequence
    returns- True if mignotte_seq is a mignotte sequence and False otherwise. Also returns the lower and upper bound of the secret
    '''
    p1=1
    for i in range(0,k):
        p1*=mignotte_seq[i]
    p2=1
    for i in range(n-k+1,n):
        p2*=mignotte_seq[i]
    return((p1>p2,p1,p2)) 

def generate_key(lower_bound,upper_bound):
    '''
    params: The lower and upper limit of the range of values the secret can take
    returns: A random integer within the range (lower_bound, upper_bound) i.e the secret
    '''
    return(random.randint(lower_bound+1,upper_bound-1))

def split_key(S,M):
    '''
    params- S: the secret, 
            M: the mignotte sequence
    returns- The shares dsitributed to each of n users
    '''
    shares=[]
    for i in M:
        shares.append(S%i)
    return(shares)

def extended_euclid(a,b):
    '''
    Find the gcd of params a,b and computes integers x,y such that ax+by=gcd(a,b)
    '''
    if(b==0):
        return(a,1,0)
    d,x,y=extended_euclid(b,a%b)
    return((d,y,x-(a//b)*y))

def chinese_remainder_theorem(M,shares,k):
    '''
    params- M: the primes corresponding to the shares available, 
            shares: the shares available,
            k: Number of shares available
    returns- The unique solution modulo prod(M) to the system of equations described by y=shares_i(mod M_i)
    '''
    if(len(shares)<k):
        return(-1)
    prod=reduce(lambda x,y: x*y, M)
    m=[prod//M[i] for i in range(0,k)]
    c=[extended_euclid(m[i],M[i])[1]%M[i] for i in range(0,k)]
    A=[c[i]*shares[i]*m[i] for i in range(k)]
    return(sum(A)%prod)


### A (10,3) Mignotte Secret Sharing Example

In [3]:
M=[]
n=10
k=3

print("(n,k)",(n,k))
mignotte_condition=False
start=1000

while(not(mignotte_condition)):
    start+=1
    M=generate_primes(start,n)
    mignotte_condition,upper_bound,lower_bound=check_mignotte_sequence(n,k,M)

S=generate_key(lower_bound,upper_bound)
print("Secret: ",S)
print("Mignotte Sequence: ")
print(M)
shares=split_key(S,M)

share_no=list(range(n))
available_shares=random.sample(share_no,k)

M_s=[M[i] for i in available_shares]
s=[shares[i] for i in available_shares]
print("Shares: ")
print(shares)
# print(chinese_remainder_theorem(M[0:k],shares[0:k],k))
print("Recovered Secret :",chinese_remainder_theorem(M_s,s,k))

(n,k) (10, 3)
Secret:  130746654
Mignotte Sequence: 
[1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061]
Shares: 
[434, 770, 802, 457, 389, 877, 972, 343, 152, 685]
Recovered Secret : 130746654
